In [1]:
import datasets as ds
import transformers as tr

In [2]:
dataset = ds.load_dataset("nllg/mt-metric-synth-plus", split='train')

In [3]:
def format_input_ref(example):
    src = example['src']
    ref = example['ref']
    mt = example['mt']
    lang1, lang2 = example['lp'].split("-")
    return {'input': f"Translation quality of: {lang1} Source: {src}; {lang2} Reference: {ref}; {lang2} Translation: {mt}"}

In [4]:
def format_input_qe(example):
    src = example['src']
    mt = example['mt']
    lang1, lang2 = example['lp'].split("-")
    return {'input': f"Translation quality of: {lang1} Source: {src}; {lang2} Translation: {mt}"}

In [5]:
dataset_ref = dataset.map(
    format_input_ref, 
    num_proc=10,
    remove_columns=[
        'ref', 'src', 'mt',
        'lp', 'system', 'example-id',
        'unit-id', 'score', 'annotations',
        '__index_level_0__', '_tokens'
    ]
)

Map (num_proc=10):   0%|          | 0/13201418 [00:00<?, ? examples/s]

In [6]:
dataset_qe = dataset.map(
    format_input_qe, 
    num_proc=10, 
    remove_columns=[
        'ref', 'src', 'mt',
        'lp', 'system', 'example-id',
        'unit-id', 'score', 'annotations',
        '__index_level_0__', '_tokens'
    ]
)

Map (num_proc=10):   0%|          | 0/13201418 [00:00<?, ? examples/s]

In [7]:
dataset = ds.concatenate_datasets([dataset_qe, dataset_ref]).shuffle(seed=0)

In [8]:
!rm -rf ./data && mkdir ./data

In [9]:
dataset.to_json(
    './data/dataset.jsonl.zst',
    batch_size=1024,
    num_proc=40,
    compression='infer'
)

Creating json from Arrow format:   0%|          | 0/25785 [00:00<?, ?ba/s]

24722743784

In [23]:
from model_utils import DataCollator1Way, DataCollator2Way, DataCollator3Way

In [43]:
collator = DataCollator3Way(tokenizer, padding=True)

In [48]:
subset = dataset.select(range(1000))

In [49]:
subset = subset.map(tokenize_function_3way, batched=True, batch_size=32, remove_columns=subset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [50]:
from torch.utils.data import DataLoader

In [51]:
dataloader = DataLoader(subset, batch_size=2, collate_fn=collator)

In [52]:
x = next(iter(dataloader))

In [54]:
x['antiexpl']['input_ids'].shape

torch.Size([2, 50])